In [1]:
from langchain_community.document_loaders import PyPDFLoader

In [2]:
pdf_path = "../data/test_document.pdf"

In [3]:
loader = PyPDFLoader(pdf_path)
documents = loader.load()

In [4]:
print(f"Документ успешно загружен.")
print(f"Тип объекта, который мы получили: {type(documents)}")
print(f"Количество элементов в списке: {len(documents)}")
print(f"Тип одного элемента: {type(documents[0])}")

Документ успешно загружен.
Тип объекта, который мы получили: <class 'list'>
Количество элементов в списке: 25
Тип одного элемента: <class 'langchain_core.documents.base.Document'>


In [5]:
first_page_document = documents[0]

print("\n--- МЕТАДАННЫЕ ПЕРВОЙ СТРАНИЦЫ ---")
print(first_page_document.metadata)

print("\n--- СОДЕРЖИМОЕ (КОНТЕНТ) ПЕРВОЙ СТРАНИЦЫ ---")
print(first_page_document.page_content)


--- МЕТАДАННЫЕ ПЕРВОЙ СТРАНИЦЫ ---
{'producer': 'GPL Ghostscript 10.02.1', 'creator': 'PDF24 Creator', 'creationdate': '2025-06-23T11:47:28+03:00', 'moddate': '2025-06-23T11:47:28+03:00', 'title': '<CFEEF1F2E0EDEEE2EBE5EDE8E520CFF0E0E2E8F2E5EBFCF1F2E2E020D0D420>', 'author': 'petr.baldaev', 'source': '../data/test_document.pdf', 'total_pages': 25, 'page': 0, 'page_label': '1'}

--- СОДЕРЖИМОЕ (КОНТЕНТ) ПЕРВОЙ СТРАНИЦЫ ---
Постановление Правительства РФ от 24.12.2021 N 2464 (ред. от 12.06.2024) "О порядке обучения по охране труда и проверки знания требований охраны труда" (вместе с 
"Правилами обучения по охране труда и проверки знания требований охраны труда") 
ПРАВИТЕЛЬСТВО РОССИЙСКОЙ ФЕДЕРАЦИИ 
ПОСТАНОВЛЕНИЕ 
от 24 декабря 2021 г. N 2464 
О ПОРЯДКЕ 
ОБУЧЕНИЯ ПО ОХРАНЕ ТРУДА И ПРОВЕРКИ ЗНАНИЯ ТРЕБОВАНИЙ 
ОХРАНЫ ТРУДА 
Список изменяющих документов 
(в ред. Постановлений Правительства РФ от 30.12.2022 N 2540,
от 12.06.2024 N 792)
В соответствии с частью третьей статьи 219 Трудовог о код

In [6]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [7]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len
)

In [8]:
chunks = text_splitter.split_documents(documents)

In [9]:
print(f"Было {len(documents)} страниц, а стало {len(chunks)} чанков.")

print("\n--- Пример первого чанка ---")
# Убедимся, что чанк существует, прежде чем выводить его
if len(chunks) > 0:
    print(chunks[0].page_content)
    print(f"\nДлина: {len(chunks[0].page_content)} символов")
    print(f"Метаданные: {chunks[0].metadata}")

print("\n--- Пример второго чанка ---")
# То же самое для второго чанка
if len(chunks) > 1:
    print(chunks[1].page_content)
    print(f"\nДлина: {len(chunks[1].page_content)} символов")
    print(f"Метаданные: {chunks[1].metadata}")

Было 25 страниц, а стало 133 чанков.

--- Пример первого чанка ---
Постановление Правительства РФ от 24.12.2021 N 2464 (ред. от 12.06.2024) "О порядке обучения по охране труда и проверки знания требований охраны труда" (вместе с 
"Правилами обучения по охране труда и проверки знания требований охраны труда") 
ПРАВИТЕЛЬСТВО РОССИЙСКОЙ ФЕДЕРАЦИИ 
ПОСТАНОВЛЕНИЕ 
от 24 декабря 2021 г. N 2464 
О ПОРЯДКЕ 
ОБУЧЕНИЯ ПО ОХРАНЕ ТРУДА И ПРОВЕРКИ ЗНАНИЯ ТРЕБОВАНИЙ 
ОХРАНЫ ТРУДА 
Список изменяющих документов 
(в ред. Постановлений Правительства РФ от 30.12.2022 N 2540,
от 12.06.2024 N 792)
В соответствии с частью третьей статьи 219 Трудовог о кодекса Российской Федерации 
Правительство Российской Федерации постановляет: 
1. Утвердить прилагаемые Правила обучения по охране т руда и проверки знания требований охраны 
труда (далее - Правила). 
2. Установить, что: 
положения пункта 78 Правил применяются с 1 марта 2023 г.; 
положения пункта 99 Правил в части, касающейся осущес твления работодателем деят

In [11]:
import os
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma

In [12]:
load_dotenv()

# Проверяем, что ключ успешно загрузился
if "OPENAI_API_KEY" not in os.environ:
    raise ValueError("Не найден OPENAI_API_KEY. Убедитесь, что он есть в файле .env")

In [13]:
embeddings_model = OpenAIEmbeddings(model="text-embedding-3-small")

In [15]:
# Указываем папку, куда будет сохранена база данных
persist_directory = 'chroma_db'

# Создаем базу данных
db = Chroma.from_documents(
    documents=chunks,
    embedding=embeddings_model,
    persist_directory=persist_directory
)

In [16]:
print("База данных успешно создана и сохранена.")
print(f"В ней содержится {db._collection.count()} документов.")

База данных успешно создана и сохранена.
В ней содержится 133 документов.


In [17]:
# Указываем ту же папку, что и при сохранении
persist_directory = 'chroma_db'

# Инициализируем модель эмбеддингов (она нужна для векторизации нашего вопроса)
embeddings_model = OpenAIEmbeddings(model="text-embedding-3-small")

# Загружаем базу данных с диска
db = Chroma(
    persist_directory=persist_directory,
    embedding_function=embeddings_model
)

print("База данных успешно загружена с диска.")

База данных успешно загружена с диска.


/var/folders/zg/vd4zt9bj0zn_ybxbpj3n39x80000gn/T/ipykernel_73600/751015001.py:8: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  db = Chroma(
